In [4]:
import math
import numpy as np
from numba import njit


@njit
def inverse_tiny(x: np.float64, tiny: np.float64 = np.float64(1e-30)) -> np.float64:
    if -tiny < x < tiny:
        x = tiny
    return np.float64(1.0)/x

@njit
def incomplete_beta_cdf_continued_fraction(\
    x: np.float64, a: np.uint32, b: np.uint32,\
    max_iter: np.uint32 = np.uint32(2000),\
    eps: np.float64 = np.float64(1e-20)) -> np.float64:
    """
    Returns the continued-fraction part of I_x(a,b), i.e. the factor
    that multiplies the 'front' = (x^a * (1-x)^b) / [a * B(a,b)].
    For large x (or after symmetry), the incomplete Beta integral is:
        I_x(a,b) = front * incomplete_beta_cdf_continued_fraction(a,b,x).
    Lentz's method is used to evaluate the CF.
    """
    # reference: https://github.com/codeplea/incbeta/blob/master/incbeta.c
    # The continued fraction converges nicely for x < (a+1)/(a+b+2)
    if x > (a+np.float64(1.0))/(a+b+np.float64(2.0)):
        # Use the fact that beta is symmetrical.
        return 1.0-incomplete_beta_cdf_continued_fraction(np.float64(1.0)-x,b,a,max_iter,eps) 
    # Find the first part before the continued fraction.
    lbeta_ab = math.lgamma(a)+math.lgamma(b)-math.lgamma(a+b)
    front = math.exp(math.log(x)*a+math.log(1.0-x)*b-lbeta_ab) / a
    
    # Setup
    tiny = np.float64(1e-30)
    # Use Lentz's algorithm to evaluate the continued fraction.
    f = np.float64(1.0)
    c = np.float64(1.0)
    d = np.float64(0.0)

    for i in range(max_iter+1):
        m = i // 2
        if i == 0:
            numerator = np.float64(1.0) # First numerator is 1.0
        elif i % 2 == 0:
            numerator = (m*(b-m)*x)/((a+2.0*m-1.0)*(a+2.0*m)) # Even term
        else:
            numerator = -((a+m)*(a+b+m)*x)/((a+2.0*m)*(a+2.0*m+1)); # Odd term
        # Do an iteration of Lentz's algorithm
        d = np.float64(1.0) + numerator * d
        d = inverse_tiny(d, tiny=tiny)

        c = np.float64(1.0) + numerator * inverse_tiny(c, tiny=tiny)
        cd = c*d
        f *= cd

        # Check for stop
        if abs(1.0-cd) < eps:
            return front * (f-1.0)
    return np.float64(-1.0) # did not converge!!!


@njit
def bark_jones_transform_numba(p_values,\
                               max_iter: np.uint32 = np.uint32(2000),\
                               eps: np.float64 = np.float64(1e-20)):
    """
    Numba-jitted Bark-Jones transform on a sorted list of p-values.
    For the i-th p-value in ascending order (p[i]), we compute:
        T_i = BetaCDF(p[i], i, n-i+1)
    where BetaCDF is the regularized incomplete beta function.
    
    Parameters
    ----------
    p_values : 1D np.ndarray of floats
        Sorted p-values in ascending order.
        
    Returns
    -------
    np.ndarray of floats
        The Bark-Jones-transformed p-values.
    """
    n = p_values.shape[0]
    transformed = np.empty(n, dtype=np.float32)
    for i in range(n):
        # The rank is i+1
        a = np.uint32(i + 1)
        b = n - i  # = n - (i+1) + 1
        x = p_values[i]
        transformed[i] = incomplete_beta_cdf_continued_fraction(x, a, b, max_iter=max_iter, eps=eps)
    return transformed

def example_usage(pvals):
    # Example usage on CPU
    n = np.uint32(len(pvals))
    transformed = bark_jones_transform_numba(pvals)
    print("Original p-values:   ", pvals)
    print("Bark-Jones numba njit transform:", transformed)


import numpy as np
from scipy.stats import beta

def bark_jones_transform(p_values):
    """
    Apply the Bark-Jones transformation to a sorted list/array of p-values.
    
    Parameters
    ----------
    p_values : array_like
        A 1D array (or list) of p-values in ascending (sorted) order.
        
    Returns
    -------
    transformed : np.ndarray
        The Bark-Jones-transformed p-values, where each element is
        beta.cdf(p, i+1, n-i), with i = 0..n-1 for the sorted p-values.
    """
    p_values = np.asarray(p_values)
    n = len(p_values)
    transformed = np.empty_like(p_values, dtype=float)
    for i, p in enumerate(p_values):
        # i is the rank, matching the sorted index
        transformed[i] = beta.cdf(p, i+1, n - i)
    
    return transformed

def example_usage_scipy(pvals):
    transformed = bark_jones_transform(pvals)
    print("Original p-values:   ", pvals)
    print("Bark-Jones scipy transform:     ", transformed)

if __name__ == "__main__":
    pvals = np.array([0.01, 0.02, 0.05, 0.10, 0.20,0.99])  # sorted
    example_usage(pvals)
    example_usage_scipy(pvals)

Original p-values:    [0.01 0.02 0.05 0.1  0.2  0.99]
Bark-Jones numba njit transform: [0.05851985 0.00568712 0.00222984 0.00127    0.0016     0.94148016]
Original p-values:    [0.01 0.02 0.05 0.1  0.2  0.99]
Bark-Jones scipy transform:      [0.05851985 0.00568712 0.00222984 0.00127    0.0016     0.94148015]
